In [7]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[1] "indathon-round1-2024"

In [ ]:
# Load necessary libraries
install.packages("readr")
install.packages("dplyr")
install.packages("lubridate")
install.packages("caret")
install.packages("randomForest")
install.packages("forecast")
install.packages("nnet")
install.packages("tidyverse")
install.packages("zoo")
library(readr)
library(dplyr)
library(lubridate)
library(caret)
library(randomForest)
library(forecast)
library(nnet)
library(zoo)
library(tidyverse)

# Load data from CSV file
data <- read.csv("penumpangtj.csv")

# Convert date column to Date format if there is a date column
# data$date <- as.Date(data$date)

# Create a time series object
ts_data <- ts(data$jumlah_penumpang, start = c(2015, 1), end = c(2023, 12), frequency = 12)

# Plot the time series
plot(ts_data, main = "Monthly Passenger Data from 2015 to 2023", xlab = "Year", ylab = "Jumlah Penumpang")

# Split data into training and testing sets (80% training, 20% testing)
train_size <- floor(0.8 * length(ts_data))
train_ts <- window(ts_data, end = c(2015 + (train_size-1)%/%12, (train_size-1)%%12+1))
test_ts <- window(ts_data, start = c(2015 + train_size%/%12, train_size%%12+1))

# Train multiple forecasting models

# Model 1: Linear Regression
train_data <- data.frame(time = time(train_ts), Passengers = as.numeric(train_ts))
model_lm <- lm(Passengers ~ time, data = train_data)

# Model 2: Random Forest
model_rf <- randomForest(Passengers ~ time, data = train_data)

# Model 3: ARIMA
model_arima <- auto.arima(train_ts)

# Model 4: Neural Network
model_nn <- nnetar(train_ts)

# Combine predictions from all models for testing data
time_test <- time(test_ts)
predictions_lm <- predict(model_lm, newdata = data.frame(time = time_test))
predictions_rf <- predict(model_rf, newdata = data.frame(time = time_test))
predictions_arima <- forecast(model_arima, h = length(test_ts))$mean
predictions_nn <- forecast(model_nn, h = length(test_ts))$mean

# Create a data frame with all predictions
ensemble_predictions <- data.frame(
  lm = predictions_lm,
  rf = predictions_rf,
  arima = predictions_arima,
  nn = predictions_nn
)

# Calculate the final ensemble prediction by averaging the predictions
final_predictions <- rowMeans(ensemble_predictions)

# Calculate RMSE for each model
rmse_lm <- sqrt(mean((predictions_lm - as.numeric(test_ts))^2))
rmse_rf <- sqrt(mean((predictions_rf - as.numeric(test_ts))^2))
rmse_arima <- sqrt(mean((predictions_arima - as.numeric(test_ts))^2))
rmse_nn <- sqrt(mean((predictions_nn - as.numeric(test_ts))^2))
rmse_ensemble <- sqrt(mean((final_predictions - as.numeric(test_ts))^2))

cat("RMSE for Linear Regression: ", rmse_lm, "\n")
cat("RMSE for Random Forest: ", rmse_rf, "\n")
cat("RMSE for ARIMA: ", rmse_arima, "\n")
cat("RMSE for Neural Network: ", rmse_nn, "\n")
cat("RMSE for Ensemble Model: ", rmse_ensemble, "\n")

# Forecast for the next 6 months using the ensemble model
forecast_horizon <- 6
time_future <- seq(from = time(ts_data)[length(ts_data)] + 1/12, by = 1/12, length.out = forecast_horizon)
forecast_lm <- predict(model_lm, newdata = data.frame(time = time_future))
forecast_rf <- predict(model_rf, newdata = data.frame(time = time_future))
forecast_arima <- forecast(model_arima, h = forecast_horizon)$mean
forecast_nn <- forecast(model_nn, h = forecast_horizon)$mean

ensemble_forecast <- rowMeans(data.frame(lm = forecast_lm,  rf = forecast_rf, arima = forecast_arima,  nn = forecast_nn))

# Store the forecasted values in a data frame
forecast_result <- data.frame(date = seq.Date(from = as.Date("2024-01-01"), by = "month", length.out = forecast_horizon),  forecast = ensemble_forecast)

# Save the forecasted values to a CSV file
write.csv(forecast_result, "forecast_result.csv", row.names = FALSE)

# Plot the forecasted values
plot(forecast_result$date, forecast_result$forecast, type = 'l', col = 'blue', main = "6-Month Forecast using Ensemble Model", xlab = "Date", ylab = "Jumlah Penumpang")

# Display forecasted values
print(forecast_result)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("readr"):
“installation of package ‘readr’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

